# 第六章：预知（逐步思考）

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
# 安装OpenAI库 - 用于与GPT AI模型进行交互
%pip install openai==1.61.0

# 导入必要的Python库
import re          # 正则表达式库，用于文本模式匹配和搜索
import openai      # OpenAI官方库，用于调用GPT API
import os          # 系统库，用于读取环境变量

# 优先从环境变量读取API配置，如果没有则从IPython存储中检索
API_KEY = os.getenv('OPENAI_API_KEY')
BASE_URL = os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')
MODEL_NAME = os.getenv('OPENAI_MODEL_NAME', 'gpt-4o')

# 如果环境变量中没有API_KEY，尝试从IPython存储中获取
if not API_KEY:
    try:
        %store -r API_KEY      # 恢复API密钥
        %store -r MODEL_NAME   # 恢复模型名称
    except:
        pass

# 如果没有设置MODEL_NAME，使用默认值
if not MODEL_NAME:
    MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

# 创建OpenAI客户端实例
# 这是与GPT AI模型交互的核心接口
client = openai.OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

def get_completion(prompt: str, system_prompt="", prefill=""):
    """
    获取GPT的完成响应 - 核心辅助函数
    
    这个函数封装了与GPT API的交互逻辑，
    让我们可以方便地发送提示并获取响应。
    
    参数:
        prompt (str): 用户提示内容 - 发送给GPT的主要消息
        system_prompt (str): 系统提示（可选） - 用于设定AI的角色和行为准则
        prefill (str): 预填充文本（可选），用于引导GPT的响应开始
    
    返回:
        str: GPT的响应文本
    """
    # 构建消息列表
    messages = []
    
    # 如果有系统提示，添加系统消息
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    
    # 添加用户消息
    messages.append({"role": "user", "content": prompt})
    
    # 如果有预填充内容，添加助手消息来模拟预填充效果
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
        # 再添加一个用户消息请求继续
        messages.append({"role": "user", "content": "请继续完成响应。"})
    
    # 调用OpenAI API
    response = client.chat.completions.create(
        model=MODEL_NAME,              # 使用的模型名称（如：gpt-4o 或 deepseek-r1）
        messages=messages,             # 消息列表
        max_tokens=2000,              # 最大生成的token数量 - 控制响应长度
        temperature=0.0               # 温度参数，0表示更确定性、一致性的回答
    )
    # 返回响应的文本内容
    return response.choices[0].message.content

---

## 课程

如果有人叫醒您，然后立即开始问您几个复杂的问题，要求您立即回答，您会表现如何？可能不如给您时间**先思考答案**时表现得好。

猜猜怎么着？GPT也是一样的。

**给GPT时间逐步思考有时会让GPT更准确**，特别是对于复杂任务。然而，**只有大声思考才算数**。您不能要求GPT思考但只输出答案——在这种情况下，实际上没有发生思考。

### 示例

在下面的提示中，对于人类读者来说，很明显第二句话与第一句话是矛盾的。但**GPT对"无关"这个词的理解过于字面化**。

In [ ]:
# 提示
# 这个例子展示了GPT有时过于字面化理解文本的问题
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# 打印GPT的响应
print(get_completion(PROMPT))

为了改进GPT的响应，让我们**允许GPT在回答之前先思考**。我们通过字面上明确说明GPT应该采取的步骤来处理和思考其任务来做到这一点。结合一点角色提示，这使GPT能够更深入地理解评论。

In [ ]:
# 系统提示
# 为GPT设置角色背景，让其更好地理解电影评论的上下文
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# 提示
# 这个改进版本让GPT在给出最终答案前先考虑正反两方面的论证
# 使用XML标签组织思考过程，确保GPT考虑到所有可能的解释
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# 打印GPT的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

**GPT有时对顺序很敏感**。这个例子处于GPT理解细致入微文本能力的前沿，当我们交换上一个例子中论证的顺序，使消极论证在前，积极论证在后时，这会改变GPT的整体评估为积极。

在大多数情况下（但令人困惑的是，并非全部），**GPT更倾向于选择两个选项中的第二个**，可能是因为在其来自网络的训练数据中，第二个选项更可能是正确的。

In [ ]:
# 提示
# 注意：这里我们交换了论证的顺序，先负面后正面
# 这个例子展示了GPT对选项顺序的敏感性
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# 打印GPT的响应
print(get_completion(PROMPT))

**让GPT思考可以将GPT的答案从错误转变为正确**。在许多GPT犯错的情况下就是这么简单！

让我们来看一个GPT答案不正确的例子，看看要求GPT思考如何解决这个问题。

In [ ]:
# 提示
# 这个例子展示GPT在没有逐步思考时可能给出错误答案
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# 打印GPT的响应
print(get_completion(PROMPT))

让我们通过要求GPT逐步思考来解决这个问题，这次使用`<brainstorm>`标签。

In [ ]:
# 提示
# 改进版本：要求GPT先在<brainstorm>标签中思考，然后给出答案
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# 打印GPT的响应
print(get_completion(PROMPT))

如果您想要在不更改上述任何内容的情况下尝试课程提示，请滚动到课程笔记本的最底部访问[**示例演练场**](#example-playground)。

---

## 练习
- [练习 6.1 - 电子邮件分类](#exercise-61---classifying-emails)
- [练习 6.2 - 电子邮件分类格式化](#exercise-62---email-classification-formatting)

### 练习 6.1 - 电子邮件分类
在本练习中，我们将指导GPT将电子邮件分类到以下类别中：										
- (A) 售前问题
- (B) 损坏或有缺陷的商品
- (C) 账单问题
- (D) 其他（请说明）

在练习的第一部分，修改`PROMPT`使**Claude输出正确的分类且仅输出分类**。您的答案需要**包括正确选择的字母（A-D），带括号，以及类别名称**。

参考`EMAILS`列表中每封邮件旁边的注释，了解该邮件应该被分类到哪个类别。

In [ ]:
# 带有占位符的提示模板
# 需要修改这个提示来正确分类电子邮件
PROMPT = """Please classify this email as either green or blue: {email}"""

# Claude响应的预填充（如果有的话）
PREFILL = ""

# 存储为列表的可变内容
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) 损坏或有缺陷的商品
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) 售前问题 或 (D) 其他（请说明）
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) 账单问题
    "How did I get here I am not good with computer.  Halp." # (D) 其他（请说明）
]

# 正确分类存储为列表的列表，以适应每封邮件可能有多个正确分类的情况
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 用于正则表达式评分的每个类别的字符串值字典
REGEX_CATEGORIES = {
    "A": "A\) P",  # (A) P 匹配 "(A) Pre-sale question" 的开头
    "B": "B\) B",  # (B) B 匹配 "(B) Broken or defective item" 的开头  
    "C": "C\) B",  # (C) B 匹配 "(C) Billing question" 的开头
    "D": "D\) O"   # (D) O 匹配 "(D) Other" 的开头
}

# 遍历电子邮件列表
for i,email in enumerate(EMAILS):
    
    # 将电子邮件文本替换到电子邮件占位符变量中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取GPT的响应
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 对GPT的响应进行评分
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # 打印GPT的响应
    print("--------------------------- 变量替换后的完整提示 ---------------------------")
    print("用户轮次")
    print(formatted_prompt)
    print("\n助手轮次")
    print(PREFILL)
    print("\n------------------------------------- GPT的响应 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 评分 ------------------------------------------")
    print("本练习已正确解决:", grade, "\n\n\n\n\n\n")

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

仍然遇到困难？运行下面的单元格查看示例解决方案。

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### 练习 6.2 - 电子邮件分类格式化
在本练习中，我们将完善上述提示的输出，以产生我们想要的确切格式的答案。

使用您最喜欢的输出格式化技术，让GPT仅将正确分类的字母包装在`<answer></answer>`标签中。例如，第一封邮件的答案应该包含确切的字符串`<answer>B</answer>`。

如果您忘记了每封邮件对应的正确字母类别，请参考`EMAILS`列表中每封邮件旁边的注释。

In [ ]:
# 带有占位符的提示模板
# 需要修改这个提示来正确分类电子邮件并使用<answer>标签格式化输出
PROMPT = """Please classify this email as either green or blue: {email}"""

# Claude响应的预填充（如果有的话）
PREFILL = ""

# 存储为列表的可变内容
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) 损坏或有缺陷的商品
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) 售前问题 或 (D) 其他（请说明）
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) 账单问题
    "How did I get here I am not good with computer.  Halp." # (D) 其他（请说明）
]

# 正确分类存储为列表的列表，以适应每封邮件可能有多个正确分类的情况
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 用于正则表达式评分的每个类别的字符串值字典
# 这次要求答案必须包含在<answer>标签中
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# 遍历电子邮件列表
for i,email in enumerate(EMAILS):
    
    # 将电子邮件文本替换到电子邮件占位符变量中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取GPT的响应
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 对GPT的响应进行评分
    # 检查响应是否包含正确的<answer>标签格式
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # 打印GPT的响应
    print("--------------------------- 变量替换后的完整提示 ---------------------------")
    print("用户轮次")
    print(formatted_prompt)
    print("\n助手轮次")
    print(PREFILL)
    print("\n------------------------------------- GPT的响应 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 评分 ------------------------------------------")
    print("本练习已正确解决:", grade, "\n\n\n\n\n\n")

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### 恭喜！

如果您已经解决了到目前为止的所有练习，那么您已经准备好进入下一章了。祝您提示工程愉快！

---

## 示例演练场

这是一个供您自由实验本课程中展示的提示示例并调整提示来观察对GPT响应影响的区域。

In [ ]:
# 提示
# 基础版本：直接询问情感倾向，不要求逐步思考
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# 打印GPT的响应
print(get_completion(PROMPT))

In [ ]:
# 系统提示
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# 提示
# 改进版本：要求GPT先考虑正面和负面论证，然后给出答案
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# 打印GPT的响应
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# 提示
# 顺序变化版本：先考虑负面论证，再考虑正面论证
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# 打印GPT的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
# 基础版本：直接提问，不要求思考过程
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# 打印GPT的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
# 改进版本：要求GPT先在<brainstorm>标签中思考演员和出生年份，然后给出答案
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# 打印GPT的响应
print(get_completion(PROMPT))